In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w6-p2/X_test.csv
/kaggle/input/2021-ai-w6-p2/train.csv
/kaggle/input/2021-ai-w6-p2/sample_submit.csv


In [2]:
train = pd.read_csv("/kaggle/input/2021-ai-w6-p2/train.csv")
test = pd.read_csv("/kaggle/input/2021-ai-w6-p2/X_test.csv")
submit = pd.read_csv("/kaggle/input/2021-ai-w6-p2/sample_submit.csv")

print(train.info())
print()
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Day     9999 non-null   int64  
 1   Hour    9999 non-null   int64  
 2   Minute  9999 non-null   int64  
 3   DHI     9999 non-null   int64  
 4   DNI     9999 non-null   int64  
 5   WS      9999 non-null   float64
 6   RH      9999 non-null   float64
 7   T       9999 non-null   int64  
 8   TARGET  9999 non-null   float64
dtypes: float64(3), int64(6)
memory usage: 703.2 KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2481 entries, 0 to 2480
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      2481 non-null   int64  
 1   Day     2481 non-null   int64  
 2   Hour    2481 non-null   int64  
 3   Minute  2481 non-null   int64  
 4   DHI     2481 non-null   int64  
 5   DNI     2481 non-null   int64  
 6   WS      2481 non-null 

In [3]:
train.head(5)

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0


In [4]:
X = train.drop(['TARGET'],axis=1)
y = train['TARGET']

X_test = test.drop(['ID'], axis = 1)

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
X_test = sc.transform(X_test)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 777)
print(X_train.shape)
print(y_train.shape)

(6999, 8)
(6999,)


In [7]:
import torch
torch.manual_seed(1)

import torch.cuda
torch.cuda.init()
torch.cuda.manual_seed(1)
torch.cuda.is_available()

True

In [8]:
X_train_tensor = torch.FloatTensor(X_train).cuda()
X_val_tensor = torch.FloatTensor(X_val).cuda()
y_train_tensor = torch.FloatTensor(y_train.to_numpy()).cuda()
y_val_tensor = torch.FloatTensor(y_val.to_numpy()).cuda()

X_test_tensor = torch.FloatTensor(X_test).cuda()

print(X_train_tensor.device)

cuda:0


In [9]:
print(X_train.shape)

(6999, 8)


In [10]:
linear1 = torch.nn.Linear(8,30, bias = True)
linear2 = torch.nn.Linear(30,30, bias = True)
linear3 = torch.nn.Linear(30,1, bias = True)
# linear4 = torch.nn.Linear(20,1, bias = True)

sigmoid = torch.nn.Sigmoid()

model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid, linear3)
model = model.cuda()

In [11]:
#가중치 랜덤
linear5 = torch.nn.Linear(8,8)
print(linear5.weight)
linear6 = torch.nn.Linear(8,8)
print(linear6.weight)

Parameter containing:
tensor([[ 0.2444,  0.2952, -0.1941,  0.2589, -0.1072, -0.0831, -0.0952, -0.0172],
        [-0.1548,  0.0484, -0.2279,  0.1454, -0.2965, -0.1282, -0.2469,  0.2385],
        [ 0.2429, -0.2812, -0.1929,  0.3164,  0.1030,  0.2434, -0.2711,  0.1596],
        [ 0.2251, -0.0820, -0.0163,  0.2578,  0.0964,  0.1014,  0.0011, -0.1476],
        [-0.2164,  0.3528, -0.0279,  0.3138,  0.2893, -0.1849,  0.0240, -0.0904],
        [-0.0064,  0.2330,  0.1947, -0.2008, -0.1749, -0.2512, -0.3002,  0.2268],
        [-0.2185,  0.0046, -0.0005,  0.0605, -0.0207,  0.1835,  0.0486,  0.2535],
        [ 0.0763, -0.0044,  0.0501,  0.2356,  0.0534,  0.2491, -0.0335,  0.0031]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.1336,  0.0801,  0.1395, -0.1517,  0.1487, -0.1121,  0.2222, -0.1038],
        [-0.3011,  0.0252,  0.1840, -0.0139,  0.0046,  0.0013,  0.1876,  0.0277],
        [-0.3158, -0.2521,  0.0209, -0.0284,  0.3379, -0.0831,  0.2706, -0.2982],
        [-0.3360, -0.0325,

In [12]:
model

Sequential(
  (0): Linear(in_features=8, out_features=30, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=30, out_features=30, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=30, out_features=1, bias=True)
)

In [13]:
def reset_parameters(m):
    m[0].reset_parameters()
    m[2].reset_parameters()
    m[4].reset_parameters()
#     m[6].reset_parameters()


In [14]:
from sklearn.metrics import mean_squared_error as MSE

reset_parameters(model)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
loss = torch.nn.MSELoss()

epochs = 20000
for epoch in range(epochs+1):
    hypothesis = model(X_train_tensor)
    cost = loss(hypothesis.squeeze(), y_train_tensor)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 1000 == 0:
        y_pred = hypothesis.cpu().detach().numpy()
        error_train = MSE(y_pred, y_train)
        y_pred = model(X_val_tensor).cpu().detach().numpy()
        error_val = MSE(y_pred, y_val)
        print('epoch:{}, cost:{:,.2f}, train_acc:{:,.3f}, val_acc:{:,.3f}'.format(epoch, cost.item(), error_train, error_val))
        
        

epoch:0, cost:1,020.80, train_acc:1,020.795, val_acc:885.228
epoch:1000, cost:6.06, train_acc:6.059, val_acc:5.704
epoch:2000, cost:2.99, train_acc:2.986, val_acc:2.934
epoch:3000, cost:1.93, train_acc:1.933, val_acc:1.915
epoch:4000, cost:1.37, train_acc:1.368, val_acc:1.352
epoch:5000, cost:1.01, train_acc:1.011, val_acc:0.995
epoch:6000, cost:0.77, train_acc:0.773, val_acc:0.757
epoch:7000, cost:0.62, train_acc:0.618, val_acc:0.605
epoch:8000, cost:0.52, train_acc:0.518, val_acc:0.509
epoch:9000, cost:0.45, train_acc:0.449, val_acc:0.446
epoch:10000, cost:0.41, train_acc:0.409, val_acc:0.411
epoch:11000, cost:0.38, train_acc:0.384, val_acc:0.388
epoch:12000, cost:0.36, train_acc:0.363, val_acc:0.369
epoch:13000, cost:0.34, train_acc:0.344, val_acc:0.353
epoch:14000, cost:0.33, train_acc:0.328, val_acc:0.338
epoch:15000, cost:0.31, train_acc:0.314, val_acc:0.325
epoch:16000, cost:0.30, train_acc:0.301, val_acc:0.314
epoch:17000, cost:0.29, train_acc:0.289, val_acc:0.303
epoch:18000, 

In [15]:
with torch.no_grad():
    hypothesis = model(X_test_tensor)
    y_pred = hypothesis.cpu().detach().numpy()
    
y_pred

array([[19.909624],
       [23.683443],
       [25.681004],
       ...,
       [11.264891],
       [11.190713],
       [11.353091]], dtype=float32)

In [16]:
submit['TARGET'] = y_pred #submit['TARGET'] = y_pred.squeeze()
submit.to_csv("submission.csv", mode = 'w', index = False)

In [17]:
submit

,ID,TARGET
0,0,19.909624
1,1,23.683443
2,2,25.681004
3,3,27.586546
4,4,28.580040
...,...,...
2476,2476,11.246636
2477,2477,11.151977
2478,2478,11.264891
2479,2479,11.190713
